### Import Dependencies

In [2]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist, euclidean

### Calculate Geometric Median for precincts in LA

In [9]:
# https://stackoverflow.com/questions/30299267/geometric-median-of-multidimensional-points
# https://www.pnas.org/content/pnas/97/4/1423.full.pdf

def geometric_median(X, eps=1e-5):
    y = np.mean(X, 0)

    while True:
        D = cdist(X, [y])
        nonzeros = (D != 0)[:, 0]

        Dinv = 1 / D[nonzeros]
        Dinvs = np.sum(Dinv)
        W = Dinv / Dinvs
        T = np.sum(W * X[nonzeros], 0)

        num_zeros = len(X) - np.sum(nonzeros)
        if num_zeros == 0:
            y1 = T
        elif num_zeros == len(X):
            return y
        else:
            R = (T - y) * Dinvs
            r = np.linalg.norm(R)
            rinv = 0 if r == 0 else num_zeros/r
            y1 = max(0, 1-rinv)*T + min(1, rinv)*y

        if euclidean(y, y1) < eps:
            return y1

        y = y1

In [14]:
def jsonFileLoader(file):
    crime_df = pd.read_json(file).transpose()
    crime_np = crime_df[['LAT','LON']].to_numpy()
    return geometric_median(crime_np)

In [19]:
def csvFileLoader(file):
    disp_df = pd.read_csv(file_to_load)["LOCATION"]
    return disp_df

In [23]:
file_to_load = '../json_files/crime_2013.json'

In [24]:
jsonFileLoader(file_to_load)

array([34.06209757390803, -118.33198449355702], dtype=object)

In [25]:
file_to_load = "../json_files/LA_Certificated_Dispensary_Locations.csv"

In [26]:
csvFileLoader(file_to_load)

0      (33.7258, -118.2855)
1      (33.7744, -118.2629)
2      (33.7805, -118.2523)
3       (33.7816, -118.243)
4      (33.7871, -118.3081)
               ...         
415     (34.307, -118.4695)
416    (34.3075, -118.4614)
417    (34.3099, -118.4274)
418    (34.3106, -118.4755)
419    (34.3106, -118.4793)
Name: LOCATION, Length: 420, dtype: object